In [1]:
import nltk
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

In [2]:
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     /home/prasannaiyer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/prasannaiyer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/prasannaiyer/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
engine = create_engine('sqlite:////home/prasannaiyer/Projects/NLP_Project/Data/DisasterResponse.db')
msg_df = pd.read_sql_table('Message_Category', engine)

In [5]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = []
    for tok in tokens:
        clean_token = lemmatizer.lemmatize(tok).lower().strip()
        lemmatized_tokens.append(clean_token)
    return lemmatized_tokens

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('MOClf', MultiOutputClassifier(estimator = KNeighborsClassifier()))
])

In [7]:
X = msg_df['message'].values


In [8]:
for column_name in msg_df.columns[~msg_df.columns.isin(['id', 'genre', 'message', 'original'])]:
    msg_df[column_name] = msg_df[column_name].astype('int')

In [9]:
y = msg_df.loc[:, ~msg_df.columns.isin(['id', 'genre', 'message', 'original'])].values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [11]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ffb34562ef0>)),
                ('tfidf', TfidfTransformer()),
                ('MOClf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

In [12]:
pred_prob = pipeline.predict_proba(X_test)

In [ ]:
pred_prob1 = pipeline.predict(X_test)

In [ ]:
classification_report(y_test, pred_prob1)

In [ ]:
y_true = np.array([[0, 1, 1, 1],[0,0,1,0],[1,1,0,0]])
y_scores = np.array([[0, 1, 1, 0],[0, 0, 1, 1],[1,1,0,1]])

In [ ]:
y_true.shape

In [ ]:
type(pred_prob1[1][1])

In [ ]:
y_train.shape

In [ ]:
label_names = ['label A', 'label B', 'label C', 'label D']

print(classification_report(y_true, y_scores))